# AI Agent 워크플로우 구현

인프런 AI 에이전트 강의를 참고하여 다음 워크플로우를 구현합니다:

```
URL 입력 → get_html → parsing_md → [요약, 태깅] → 점수 평가 → 성공/재시도
```

## 주요 기능
- **get_html**: 웹 페이지 로딩 (Playwright)
- **parsing_md**: HTML → Markdown 변환
- **요약**: 핵심 내용 추출
- **태깅**: 카테고리/태그 자동 생성
- **점수 평가**: 품질 평가 (70점 기준)
- **LangGraph**: StateGraph로 전체 플로우 관리

## 1. 환경 설정 및 패키지 Import

In [1]:
import asyncio
import os
import time
import random
from typing import Dict, List, Optional, Any, TypedDict
from urllib.parse import urlparse

# Web scraping
from playwright.async_api import async_playwright, Browser, BrowserContext, Page

# LangChain imports
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_community.document_transformers import MarkdownifyTransformer
from langchain_openai import ChatOpenAI

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Environment
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ 모든 패키지가 성공적으로 import되었습니다.")

ModuleNotFoundError: No module named 'langgraph'

## 2. 환경 변수 및 기본 설정

In [ ]:
# OpenAI API 키 확인
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")

# LLM 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    max_tokens=4000
)

print(f"✅ OpenAI LLM 초기화 완료: {llm.model_name}")

# User Agents for web scraping
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
]

✅ OpenAI LLM 초기화 완료: gpt-4o-mini


## 3. State 정의 (LangGraph용)

In [ ]:
class AgentState(TypedDict):
    """AI Agent의 상태를 정의하는 클래스"""
    url: str
    html_content: Optional[str]
    markdown_content: Optional[str]
    summary: Optional[str]
    tags: Optional[List[str]]
    summary_score: Optional[int]
    summary_reason: Optional[str]
    tagging_score: Optional[int]
    tagging_reason: Optional[str]
    retry_count: int
    error_message: Optional[str]
    final_result: Optional[Dict[str, Any]]

print("✅ AgentState 클래스 정의 완료")

✅ AgentState 클래스 정의 완료


## 4. 웹 콘텐츠 추출 함수 (get_html)

In [ ]:
async def get_html(state: AgentState) -> AgentState:
    """
    웹 페이지에서 HTML 콘텐츠를 추출하는 함수
    Playwright를 사용하여 안정적인 웹 스크래핑을 수행
    """
    url = state["url"]
    print(f"🌐 웹 페이지 로딩 시작: {url}")
    
    try:
        async with async_playwright() as p:
            # 브라우저 실행 (headless 모드)
            browser = await p.chromium.launch(
                headless=True,
                args=[
                    '--no-sandbox',
                    '--disable-dev-shm-usage',
                    '--disable-gpu',
                    '--disable-web-security',
                ]
            )
            
            # 새 페이지 생성
            context = await browser.new_context(
                user_agent=random.choice(USER_AGENTS),
                viewport={'width': 1920, 'height': 1080}
            )
            page = await context.new_page()
            
            # 페이지 로딩
            await page.goto(url, wait_until='networkidle', timeout=30000)
            
            # 페이지가 완전히 로드될 때까지 잠시 대기
            await asyncio.sleep(2)
            
            # HTML 콘텐츠 추출
            html_content = await page.content()
            
            # 브라우저 종료
            await browser.close()
            
            # 상태 업데이트
            state["html_content"] = html_content
            state["error_message"] = None
            
            print(f"✅ HTML 콘텐츠 추출 완료 (길이: {len(html_content):,} 문자)")
            return state
            
    except Exception as e:
        error_msg = f"웹 페이지 로딩 실패: {str(e)}"
        print(f"❌ {error_msg}")
        
        state["html_content"] = None
        state["error_message"] = error_msg
        return state

print("✅ get_html 함수 정의 완료")

✅ get_html 함수 정의 완료


## 5. HTML → Markdown 변환 함수 (parsing_md)

In [ ]:
def parsing_md(state: AgentState) -> AgentState:
    """
    HTML 콘텐츠를 Markdown으로 변환하는 함수
    LangChain MarkdownifyTransformer를 사용
    """
    html_content = state.get("html_content")
    
    if not html_content:
        error_msg = "HTML 콘텐츠가 없습니다."
        print(f"❌ {error_msg}")
        state["error_message"] = error_msg
        return state
    
    print("📝 HTML → Markdown 변환 시작")
    
    try:
        # LangChain Document 객체 생성
        doc = Document(page_content=html_content)
        
        # MarkdownifyTransformer 초기화 및 변환
        transformer = MarkdownifyTransformer()
        transformed_docs = transformer.transform_documents([doc])
        
        # Markdown 콘텐츠 추출
        markdown_content = transformed_docs[0].page_content
        
        # 불필요한 공백 및 줄바꿈 정리
        lines = markdown_content.split('\n')
        cleaned_lines = []
        
        for line in lines:
            line = line.strip()
            if line:  # 빈 줄이 아닌 경우만 추가
                cleaned_lines.append(line)
        
        # 정리된 Markdown 콘텐츠
        cleaned_markdown = '\n\n'.join(cleaned_lines)
        
        # 상태 업데이트
        state["markdown_content"] = cleaned_markdown
        state["error_message"] = None
        
        print(f"✅ Markdown 변환 완료 (길이: {len(cleaned_markdown):,} 문자)")
        return state
        
    except Exception as e:
        error_msg = f"Markdown 변환 실패: {str(e)}"
        print(f"❌ {error_msg}")
        
        state["markdown_content"] = None
        state["error_message"] = error_msg
        return state

print("✅ parsing_md 함수 정의 완료")

✅ parsing_md 함수 정의 완료


## 6. 요약 기능 구현 (핵심 내용 추출)

In [ ]:
from pydantic import BaseModel, Field

class SummaryGrade(BaseModel):
    """요약 품질 평가 결과"""
    score: int = Field(..., ge=0, le=100, description="0-100점 사이의 요약 품질 점수")
    reason: str = Field(..., description="점수 평가 이유")

class TaggingGrade(BaseModel):
    """태깅 품질 평가 결과"""
    score: int = Field(..., ge=0, le=100, description="0-100점 사이의 태깅 품질 점수")
    reason: str = Field(..., description="점수 평가 이유")

print("✅ SummaryGrade, TaggingGrade Pydantic 모델 정의 완료")

✅ SummaryGrade, TaggingGrade Pydantic 모델 정의 완료


In [ ]:
def summarize_content(state: AgentState) -> AgentState:
    """
    Markdown 콘텐츠의 핵심 내용을 요약하는 함수
    """
    markdown_content = state.get("markdown_content")
    
    if not markdown_content:
        error_msg = "Markdown 콘텐츠가 없습니다."
        print(f"❌ {error_msg}")
        state["error_message"] = error_msg
        return state
    
    print("📄 콘텐츠 요약 시작")
    
    try:
        # 요약 프롬프트 템플릿
        summary_prompt = ChatPromptTemplate.from_template(
            """다음 웹 페이지 콘텐츠를 분석하여 핵심 내용을 요약해주세요.
            
웹 페이지 콘텐츠:
{content}

요약 지침:
1. 주요 내용과 핵심 포인트를 3-5개의 불릿 포인트로 요약
2. 기술적 내용이 있다면 중요한 기술 스택이나 개념 포함
3. 실용적이고 유용한 정보 위주로 요약
4. 200-300자 내외로 간결하게 작성
5. 한국어로 작성

요약:"""
        )
        
        # 요약 체인 생성
        summary_chain = summary_prompt | llm | StrOutputParser()
        
        # 콘텐츠 길이 제한 (토큰 수 고려)
        max_length = 8000  # 약 2000토큰 정도
        if len(markdown_content) > max_length:
            truncated_content = markdown_content[:max_length] + "... (내용 일부 생략)"
        else:
            truncated_content = markdown_content
        
        # 요약 실행
        summary = summary_chain.invoke({"content": truncated_content})
        
        # 상태 업데이트
        state["summary"] = summary.strip()
        state["error_message"] = None
        
        print(f"✅ 요약 완료 (길이: {len(summary)} 문자)")
        print(f"📝 요약 내용: {summary[:100]}...")
        
        return state
        
    except Exception as e:
        error_msg = f"요약 생성 실패: {str(e)}"
        print(f"❌ {error_msg}")
        
        state["summary"] = None
        state["error_message"] = error_msg
        return state

print("✅ summarize_content 함수 정의 완료")

✅ summarize_content 함수 정의 완료


In [ ]:
def evaluate_summary(state: AgentState) -> AgentState:
    """
    요약의 품질을 평가하여 점수를 부여하는 함수 (0-100점)
    80점 이상이면 성공으로 판단
    """
    markdown_content = state.get("markdown_content")
    summary = state.get("summary", "")
    
    if not markdown_content or not summary:
        error_msg = "Markdown 콘텐츠 또는 요약이 없습니다."
        print(f"❌ {error_msg}")
        state["error_message"] = error_msg
        state["summary_score"] = 0
        return state
    
    print("📊 요약 품질 평가 시작")
    
    try:
        # 요약 평가 프롬프트 템플릿
        summary_eval_prompt = ChatPromptTemplate.from_template(
            """다음 원본 콘텐츠와 요약을 비교하여 요약의 품질을 0-100점으로 평가해주세요.

원본 콘텐츠 (처음 1000자):
{content_sample}

생성된 요약:
{summary}

평가 기준:
1. 핵심 내용 포함 여부 (30점): 원본의 주요 내용이 요약에 잘 반영되었는가?
2. 정확성 (25점): 요약 내용이 원본과 일치하며 왜곡이 없는가?
3. 간결성 (20점): 불필요한 내용 없이 간결하게 작성되었는가?
4. 완성도 (15점): 읽기 쉽고 이해하기 쉬운 형태인가?
5. 구조화 (10점): 논리적 순서와 구조를 가지고 있는가?

평가:"""
        )
        
        # 평가 체인 생성
        summary_eval_chain = summary_eval_prompt | llm.with_structured_output(schema=SummaryGrade)
        
        # 콘텐츠 샘플 생성 (처음 1000자)
        content_sample = markdown_content[:1000] + "..." if len(markdown_content) > 1000 else markdown_content
        
        # 요약 평가 실행
        summary_grade = summary_eval_chain.invoke({
            "content_sample": content_sample,
            "summary": summary
        })
        
        # 점수 범위 검증
        score = max(0, min(100, summary_grade.score))
        
        # 상태 업데이트
        state["summary_score"] = score
        state["summary_reason"] = summary_grade.reason
        state["error_message"] = None
        
        print(f"✅ 요약 평가 완료: {score}점")
        print(f"📝 평가 이유: {summary_grade.reason}")
        
        return state
        
    except Exception as e:
        error_msg = f"요약 평가 실패: {str(e)}"
        print(f"❌ {error_msg}")
        state["summary_score"] = 0
        state["error_message"] = error_msg
        return state

def evaluate_tagging(state: AgentState) -> AgentState:
    """
    태깅의 품질을 평가하여 점수를 부여하는 함수 (0-100점)
    80점 이상이면 성공으로 판단
    """
    markdown_content = state.get("markdown_content")
    tags = state.get("tags", [])
    
    if not markdown_content or not tags:
        error_msg = "Markdown 콘텐츠 또는 태그가 없습니다."
        print(f"❌ {error_msg}")
        state["error_message"] = error_msg
        state["tagging_score"] = 0
        return state
    
    print("🏷️ 태깅 품질 평가 시작")
    
    try:
        # 태깅 평가 프롬프트 템플릿
        tagging_eval_prompt = ChatPromptTemplate.from_template(
            """다음 원본 콘텐츠와 생성된 태그를 비교하여 태깅의 품질을 0-100점으로 평가해주세요.

원본 콘텐츠 (처음 1000자):
{content_sample}

생성된 태그:
{tags}

평가 기준:
1. 관련성 (30점): 태그가 콘텐츠의 주제와 얼마나 관련 있는가?
2. 정확성 (25점): 태그가 콘텐츠 내용을 정확히 반영하는가?
3. 적절성 (20점): 태그의 개수와 구체성이 적절한가?
4. 유용성 (15점): 검색이나 분류에 실제로 도움이 되는가?
5. 다양성 (10점): 다양한 관점의 태그를 포함하고 있는가?

평가:"""
        )
        
        # 평가 체인 생성
        tagging_eval_chain = tagging_eval_prompt | llm.with_structured_output(schema=TaggingGrade)
        
        # 콘텐츠 샘플 생성 (처음 1000자)
        content_sample = markdown_content[:1000] + "..." if len(markdown_content) > 1000 else markdown_content
        
        # 태깅 평가 실행
        tagging_grade = tagging_eval_chain.invoke({
            "content_sample": content_sample,
            "tags": ', '.join(tags)
        })
        
        # 점수 범위 검증
        score = max(0, min(100, tagging_grade.score))
        
        # 상태 업데이트
        state["tagging_score"] = score
        state["tagging_reason"] = tagging_grade.reason
        state["error_message"] = None
        
        print(f"✅ 태깅 평가 완료: {score}점")
        print(f"📝 평가 이유: {tagging_grade.reason}")
        
        return state
        
    except Exception as e:
        error_msg = f"태깅 평가 실패: {str(e)}"
        print(f"❌ {error_msg}")
        state["tagging_score"] = 0
        state["error_message"] = error_msg
        return state

print("✅ evaluate_summary, evaluate_tagging 함수 정의 완료")

✅ evaluate_summary, evaluate_tagging 함수 정의 완료


## 7. 태깅 기능 구현 (카테고리/태그 생성)

In [ ]:
def generate_tags(state: AgentState) -> AgentState:
    """
    Markdown 콘텐츠에서 관련 태그를 자동 생성하는 함수
    """
    markdown_content = state.get("markdown_content")
    summary = state.get("summary", "")
    
    if not markdown_content:
        error_msg = "Markdown 콘텐츠가 없습니다."
        print(f"❌ {error_msg}")
        state["error_message"] = error_msg
        return state
    
    print("🏷️ 태그 생성 시작")
    
    try:
        # 태깅 프롬프트 템플릿
        tagging_prompt = ChatPromptTemplate.from_template(
            """다음 웹 페이지 콘텐츠를 분석하여 관련 태그를 생성해주세요.

웹 페이지 콘텐츠:
{content}

요약 내용:
{summary}

태그 생성 지침:
1. 콘텐츠의 주제, 기술 스택, 분야를 반영한 태그 생성
2. 3-7개의 태그를 생성 (너무 많지 않게)
3. 각 태그는 2-15자 내외의 간결한 단어 또는 구문
4. 기술 관련 내용의 경우: 언어, 프레임워크, 도구명 포함
5. 일반 내용의 경우: 주제, 분야, 카테고리 포함
6. 한국어 또는 영어 태그 모두 가능 (내용에 따라 적절히)
7. JSON 배열 형태로 응답: [\"태그1\", \"태그2\", \"태그3\"]

태그:"""
        )
        
        # JSON 파서 설정
        json_parser = JsonOutputParser()
        
        # 태깅 체인 생성
        tagging_chain = tagging_prompt | llm | json_parser
        
        # 콘텐츠 길이 제한 (토큰 수 고려)
        max_length = 6000  # 약 1500토큰 정도
        if len(markdown_content) > max_length:
            truncated_content = markdown_content[:max_length] + "... (내용 일부 생략)"
        else:
            truncated_content = markdown_content
        
        # 태그 생성 실행
        tags_result = tagging_chain.invoke({
            "content": truncated_content,
            "summary": summary
        })
        
        # 결과 처리
        if isinstance(tags_result, list):
            tags = tags_result
        else:
            # 백업: 문자열로 반환된 경우 파싱 시도
            import json
            tags = json.loads(tags_result) if isinstance(tags_result, str) else []
        
        # 태그 검증 및 정리
        cleaned_tags = []
        for tag in tags:
            if isinstance(tag, str) and 2 <= len(tag.strip()) <= 20:
                cleaned_tags.append(tag.strip())
        
        # 최대 7개로 제한
        if len(cleaned_tags) > 7:
            cleaned_tags = cleaned_tags[:7]
        
        # 상태 업데이트
        state["tags"] = cleaned_tags
        state["error_message"] = None
        
        print(f"✅ 태그 생성 완료: {cleaned_tags}")
        
        return state
        
    except Exception as e:
        error_msg = f"태그 생성 실패: {str(e)}"
        print(f"❌ {error_msg}")
        
        # 백업 태그 생성
        backup_tags = ["웹콘텐츠", "정보"]
        state["tags"] = backup_tags
        state["error_message"] = error_msg
        
        print(f"🔄 백업 태그 사용: {backup_tags}")
        return state

print("✅ generate_tags 함수 정의 완료")

✅ generate_tags 함수 정의 완료


# 기존 evaluate_score 함수는 제거되고 evaluate_summary와 evaluate_tagging으로 분리되었습니다.
print("✅ 평가 시스템 개선 완료: 요약과 태깅을 별도로 평가하여 각각 80점 이상이어야 성공")

In [ ]:
# 유틸리티 함수들
def should_retry(state: AgentState) -> str:
    """
    요약과 태깅 점수가 모두 80점 이상이고 재시도 횟수가 3회 미만인 경우 재시도 결정
    """
    summary_score = state.get("summary_score", 0)
    tagging_score = state.get("tagging_score", 0)
    retry_count = state.get("retry_count", 0)
    
    print(f"📊 평가 결과 - 요약: {summary_score}점, 태깅: {tagging_score}점")
    
    # 두 점수 모두 80점 이상이면 성공
    if summary_score >= 80 and tagging_score >= 80:
        return "success"
    # 재시도 횟수가 3회 미만이면 재시도
    elif retry_count < 3:
        return "retry"
    # 재시도 횟수 초과 시 실패
    else:
        return "fail"

def increment_retry(state: AgentState) -> AgentState:
    """재시도 횟수 증가"""
    state["retry_count"] = state.get("retry_count", 0) + 1
    print(f"🔄 재시도 {state['retry_count']}/3회")
    return state

def create_final_result(state: AgentState) -> AgentState:
    """최종 결과 생성"""
    summary_score = state.get("summary_score", 0)
    tagging_score = state.get("tagging_score", 0)
    
    # 두 점수 모두 80점 이상이어야 성공
    success = summary_score >= 80 and tagging_score >= 80
    
    final_result = {
        "url": state["url"],
        "success": success,
        "summary_score": summary_score,
        "summary_reason": state.get("summary_reason"),
        "tagging_score": tagging_score,
        "tagging_reason": state.get("tagging_reason"),
        "summary": state.get("summary"),
        "tags": state.get("tags", []),
        "retry_count": state.get("retry_count", 0),
        "error_message": state.get("error_message")
    }
    
    state["final_result"] = final_result
    
    if success:
        print(f"🎉 워크플로우 성공 완료! (요약: {summary_score}점, 태깅: {tagging_score}점)")
    else:
        print(f"😞 워크플로우 실패 (요약: {summary_score}점, 태깅: {tagging_score}점, 재시도: {state.get('retry_count', 0)}회)")
    
    return state

print("✅ 유틸리티 함수들 정의 완료")

✅ 유틸리티 함수들 정의 완료


In [ ]:
# LangGraph StateGraph 생성
def create_ai_agent_workflow():
    """AI Agent 워크플로우 생성"""
    
    # StateGraph 초기화
    workflow = StateGraph(AgentState)
    
    # 노드 추가
    workflow.add_node("get_html", get_html)
    workflow.add_node("parsing_md", parsing_md) 
    workflow.add_node("summarize", summarize_content)
    workflow.add_node("tagging", generate_tags)
    workflow.add_node("evaluate_summary", evaluate_summary)
    workflow.add_node("evaluate_tagging", evaluate_tagging)
    workflow.add_node("increment_retry", increment_retry)
    workflow.add_node("finalize", create_final_result)
    
    # 시작점 설정
    workflow.set_entry_point("get_html")
    
    # 엣지 설정
    workflow.add_edge("get_html", "parsing_md")
    workflow.add_edge("parsing_md", "summarize")
    workflow.add_edge("summarize", "tagging") 
    workflow.add_edge("tagging", "evaluate_summary")
    workflow.add_edge("evaluate_summary", "evaluate_tagging")
    
    # 조건부 엣지 (두 점수 모두 평가 후 분기)
    workflow.add_conditional_edges(
        "evaluate_tagging",
        should_retry,
        {
            "success": "finalize",        # 두 점수 모두 80점 이상: 성공
            "retry": "increment_retry",   # 80점 미만 & 재시도 < 3: 재시도
            "fail": "finalize"           # 80점 미만 & 재시도 >= 3: 실패
        }
    )
    
    # 재시도 시 summarize로 돌아가기 (요약과 태깅 모두 다시 실행)
    workflow.add_edge("increment_retry", "summarize")
    
    # 종료점 설정
    workflow.add_edge("finalize", END)
    
    # 메모리 체크포인트 추가
    memory = MemorySaver()
    
    # 컴파일
    app = workflow.compile(checkpointer=memory)
    
    return app

# 워크플로우 생성
ai_agent_app = create_ai_agent_workflow()

print("✅ AI Agent 워크플로우 생성 완료!")

✅ AI Agent 워크플로우 생성 완료!


# 기존 워크플로우 함수는 제거되고 cell-19의 새로운 구조로 통일되었습니다.
print("✅ 워크플로우 구조 통일 완료: evaluate_summary + evaluate_tagging 사용")

In [ ]:
# LangGraph StateGraph 생성
def create_ai_agent_workflow():
    """AI Agent 워크플로우 생성"""
    
    # StateGraph 초기화
    workflow = StateGraph(AgentState)
    
    # 노드 추가
    workflow.add_node("get_html", get_html)
    workflow.add_node("parsing_md", parsing_md) 
    workflow.add_node("summarize", summarize_content)
    workflow.add_node("tagging", generate_tags)
    workflow.add_node("evaluate_summary", evaluate_summary)
    workflow.add_node("evaluate_tagging", evaluate_tagging)
    workflow.add_node("increment_retry", increment_retry)
    workflow.add_node("finalize", create_final_result)
    
    # 시작점 설정
    workflow.set_entry_point("get_html")
    
    # 엣지 설정
    workflow.add_edge("get_html", "parsing_md")
    workflow.add_edge("parsing_md", "summarize")
    workflow.add_edge("summarize", "tagging") 
    workflow.add_edge("tagging", "evaluate_summary")
    workflow.add_edge("evaluate_summary", "evaluate_tagging")
    
    # 조건부 엣지 (두 점수 모두 평가 후 분기)
    workflow.add_conditional_edges(
        "evaluate_tagging",
        should_retry,
        {
            "success": "finalize",        # 두 점수 모두 80점 이상: 성공
            "retry": "increment_retry",   # 80점 미만 & 재시도 < 3: 재시도
            "fail": "finalize"           # 80점 미만 & 재시도 >= 3: 실패
        }
    )
    
    # 재시도 시 summarize로 돌아가기 (요약과 태깅 모두 다시 실행)
    workflow.add_edge("increment_retry", "summarize")
    
    # 종료점 설정
    workflow.add_edge("finalize", END)
    
    # 메모리 체크포인트 추가
    memory = MemorySaver()
    
    # 컴파일
    app = workflow.compile(checkpointer=memory)
    
    return app

# 워크플로우 생성
ai_agent_app = create_ai_agent_workflow()

print("✅ AI Agent 워크플로우 생성 완료!")

✅ AI Agent 워크플로우 생성 완료!


In [ ]:
# 기존 run_ai_agent 함수는 제거되고 cell-21의 새로운 버전으로 통일되었습니다.
print("✅ run_ai_agent 함수 통일 완료: 요약과 태깅 점수 표시 지원")

✅ run_ai_agent 함수 통일 완료: 요약과 태깅 점수 표시 지원


In [ ]:
async def run_ai_agent(url: str) -> Dict[str, Any]:
    """
    AI Agent 워크플로우를 실행하는 메인 함수
    
    Args:
        url: 분석할 웹페이지 URL
        
    Returns:
        최종 결과 딕셔너리
    """
    
    print(f"🚀 AI Agent 워크플로우 시작: {url}")
    print("=" * 60)
    
    # 초기 상태 설정
    initial_state = {
        "url": url,
        "html_content": None,
        "markdown_content": None,
        "summary": None,
        "tags": None,
        "summary_score": None,
        "summary_reason": None,
        "tagging_score": None,
        "tagging_reason": None,
        "retry_count": 0,
        "error_message": None,
        "final_result": None
    }
    
    # 스레드 설정 (메모리 관리용)
    thread_config = {"configurable": {"thread_id": f"ai_agent_{int(time.time())}"}}
    
    try:
        # 워크플로우 실행 - astream 사용 (async 함수 때문에)
        final_state = None
        async for step in ai_agent_app.astream(initial_state, thread_config):
            print(f"📋 현재 단계: {list(step.keys())[0]}")
            final_state = step
        
        # 최종 결과 추출 - finalize 노드에서 가져오기
        if final_state and "finalize" in final_state:
            result = final_state["finalize"]["final_result"]
        else:
            # 백업: 마지막 state에서 직접 가져오기
            last_node_key = list(final_state.keys())[0]
            result = final_state[last_node_key]["final_result"]
        
        print("=" * 60)
        print("🎯 최종 결과:")
        print(f"   • URL: {result['url']}")
        print(f"   • 성공 여부: {'✅ 성공' if result['success'] else '❌ 실패'}")
        print(f"   • 요약 점수: {result['summary_score']}/100점")
        if result['summary_reason']:
            print(f"   • 요약 평가: {result['summary_reason']}")
        print(f"   • 태깅 점수: {result['tagging_score']}/100점")
        if result['tagging_reason']:
            print(f"   • 태깅 평가: {result['tagging_reason']}")
        print(f"   • 재시도 횟수: {result['retry_count']}회")
        print(f"   • 태그: {', '.join(result['tags']) if result['tags'] else '없음'}")
        if result['summary']:
            print(f"""
   • 요약: 
{result['summary']}
                        """)
        if result['error_message']:
            print(f"   • 오류: {result['error_message']}")
        
        return result
        
    except Exception as e:
        error_result = {
            "url": url,
            "success": False,
            "summary_score": 0,
            "summary_reason": None,
            "tagging_score": 0,
            "tagging_reason": None,
            "summary": None,
            "tags": [],
            "retry_count": 0,
            "error_message": f"워크플로우 실행 오류: {str(e)}"
        }
        
        print(f"❌ 워크플로우 실행 실패: {str(e)}")
        return error_result

print("✅ run_ai_agent 함수 정의 완료")

✅ run_ai_agent 함수 정의 완료


In [ ]:
# 중복된 run_ai_agent 함수가 제거되었습니다. cell-23의 올바른 버전을 사용합니다.
print("✅ run_ai_agent 함수 정리 완료: 새로운 구조(summary_score, tagging_score) 사용")

✅ run_ai_agent 함수 정리 완료: 새로운 구조(summary_score, tagging_score) 사용


In [ ]:
result = await run_ai_agent("https://techblog.lycorp.co.jp/ko/p-canvas-a-technique-for-understanding-your-team")

🚀 AI Agent 워크플로우 시작: https://techblog.lycorp.co.jp/ko/p-canvas-a-technique-for-understanding-your-team
🌐 웹 페이지 로딩 시작: https://techblog.lycorp.co.jp/ko/p-canvas-a-technique-for-understanding-your-team
✅ HTML 콘텐츠 추출 완료 (길이: 263,145 문자)
📋 현재 단계: get_html
📝 HTML → Markdown 변환 시작
✅ Markdown 변환 완료 (길이: 18,401 문자)
📋 현재 단계: parsing_md
📄 콘텐츠 요약 시작
✅ 요약 완료 (길이: 419 문자)
📝 요약 내용: - **P-Canvas 개념**: P-Canvas는 팀의 개인 성장과 현재 상태를 시각화하는 매니징 프레임워크로, 리드와 멤버 간의 의미 있는 대화를 촉진하기 위해 설계됨.
- *...
📋 현재 단계: summarize
🏷️ 태그 생성 시작
✅ 태그 생성 완료: ['P-Canvas', '매니징 엔지니어링', '팀 관리', '1on1 미팅', '개인 성장', '데이터 기반 대화', 'ABC Platform']
📋 현재 단계: tagging
📊 요약 품질 평가 시작
✅ 요약 평가 완료: 85점
📝 평가 이유: 요약은 원본의 핵심 내용을 잘 반영하고 있으며, P-Canvas의 개념, 목적, 운영 방식, 장점, 사례를 포함하고 있습니다. 내용의 정확성도 높고, 간결하게 작성되어 있습니다. 그러나 약간의 세부사항이 생략되어 있어 완성도에서 다소 아쉬움이 있습니다.
📋 현재 단계: evaluate_summary
🏷️ 태깅 품질 평가 시작
✅ 태깅 평가 완료: 75점
📝 평가 이유: 태그는 원본 콘텐츠의 주제와 관련이 있으며, 특히 'P-Canvas'와 '팀 관리'는 핵심 개념을 잘 반영하고 있습니다. 그러나 '1on1 미팅'과 '개인 성장'은 콘텐츠에서 직접적으로 언급되지 않아 정확성에서 다소 부족합니다. 태그의 개수는

In [ ]:
result = await run_ai_agent("https://techblog.gccompany.co.kr/%EC%97%AC%ED%96%89%EB%8F%84-%ED%95%98%EA%B3%A0-%EC%A7%80%EA%B5%AC%EB%8F%84-%EC%A7%80%ED%82%A8%EB%8B%A4-%EC%97%AC%EA%B8%B0%EC%96%B4%EB%95%8C-%EC%93%B0%EB%B4%89%ED%81%AC%EB%9F%BD-%EB%94%94%EC%9E%90%EC%9D%B8-%EB%A6%AC%EB%89%B4%EC%96%BC-b14f692d9218?source=rss----18356045d353---4")

🚀 AI Agent 워크플로우 시작: https://techblog.gccompany.co.kr/%EC%97%AC%ED%96%89%EB%8F%84-%ED%95%98%EA%B3%A0-%EC%A7%80%EA%B5%AC%EB%8F%84-%EC%A7%80%ED%82%A8%EB%8B%A4-%EC%97%AC%EA%B8%B0%EC%96%B4%EB%95%8C-%EC%93%B0%EB%B4%89%ED%81%AC%EB%9F%BD-%EB%94%94%EC%9E%90%EC%9D%B8-%EB%A6%AC%EB%89%B4%EC%96%BC-b14f692d9218?source=rss----18356045d353---4
🌐 웹 페이지 로딩 시작: https://techblog.gccompany.co.kr/%EC%97%AC%ED%96%89%EB%8F%84-%ED%95%98%EA%B3%A0-%EC%A7%80%EA%B5%AC%EB%8F%84-%EC%A7%80%ED%82%A8%EB%8B%A4-%EC%97%AC%EA%B8%B0%EC%96%B4%EB%95%8C-%EC%93%B0%EB%B4%89%ED%81%AC%EB%9F%BD-%EB%94%94%EC%9E%90%EC%9D%B8-%EB%A6%AC%EB%89%B4%EC%96%BC-b14f692d9218?source=rss----18356045d353---4
✅ HTML 콘텐츠 추출 완료 (길이: 276,546 문자)
📋 현재 단계: get_html
📝 HTML → Markdown 변환 시작
✅ Markdown 변환 완료 (길이: 27,908 문자)
📋 현재 단계: parsing_md
📄 콘텐츠 요약 시작
✅ 요약 완료 (길이: 324 문자)
📝 요약 내용: - 여기어때의 쓰봉크럽은 여행과 환경 보호를 결합한 ESG 프로젝트로, 플로깅 활동을 통해 지속 가능한 여행 문화를 확산하는 것을 목표로 함.
- 최근 디자인 리뉴얼을 통해 새로운...
📋 현재 단계: summarize
🏷️ 태그 생성 시작
✅ 태그 생성 완료: ['여행', '환경 보호', '플로깅', '지속

In [ ]:
result = await run_ai_agent("https://tech.kakao.com/posts/747")

🚀 AI Agent 워크플로우 시작: https://tech.kakao.com/posts/747
🌐 웹 페이지 로딩 시작: https://tech.kakao.com/posts/747
✅ HTML 콘텐츠 추출 완료 (길이: 116,836 문자)
📋 현재 단계: get_html
📝 HTML → Markdown 변환 시작
✅ Markdown 변환 완료 (길이: 15,670 문자)
📋 현재 단계: parsing_md
📄 콘텐츠 요약 시작
✅ 요약 완료 (길이: 476 문자)
📝 요약 내용: - 카카오는 AI 기반의 운영 생태계를 구축하기 위해 자체 개발한 모니터링 솔루션 '매트릭스(MATRIX)'를 운영하고 있으며, 이를 통해 서버 및 애플리케이션의 성능을 실시간으로...
📋 현재 단계: summarize
🏷️ 태그 생성 시작
✅ 태그 생성 완료: ['AI 운영', '분산 추적', '모니터링', 'MATRIX', 'LLM', '인시던트 관리', 'OpenTelemetry']
📋 현재 단계: tagging
📊 요약 품질 평가 시작
✅ 요약 평가 완료: 85점
📝 평가 이유: 요약은 원본 콘텐츠의 핵심 내용을 잘 반영하고 있으며, AI 기반 운영 생태계와 매트릭스의 기능을 명확하게 설명하고 있습니다. 정확성 또한 높고, 간결하게 작성되어 있어 읽기 쉽습니다. 그러나 일부 세부사항이 생략되어 있어 완성도에서 약간의 아쉬움이 있습니다.
📋 현재 단계: evaluate_summary
🏷️ 태깅 품질 평가 시작
✅ 태깅 평가 완료: 85점
📝 평가 이유: 태그는 원본 콘텐츠의 주제와 관련성이 높고, 주요 개념들을 잘 반영하고 있습니다. 'AI 운영', '분산 추적', '모니터링', 'MATRIX', 'LLM' 등은 콘텐츠의 핵심 요소를 포함하고 있으며, 인시던트 관리와 OpenTelemetry도 관련성이 있습니다. 태그의 개수와 구체성도 적절하며, 검색 및 분류에 유용할 것으로 보입니다. 그러나 태그의 다양성이 다소 부족하여 완벽한 평가에는 미치지 못합니다.
📊 평가 결과 - 

In [ ]:
# 개선된 AI Agent 테스트 실행
result = await run_ai_agent("https://tech.kakao.com/posts/747")

🚀 AI Agent 워크플로우 시작: https://tech.kakao.com/posts/747
🌐 웹 페이지 로딩 시작: https://tech.kakao.com/posts/747
✅ HTML 콘텐츠 추출 완료 (길이: 116,836 문자)
📋 현재 단계: get_html
📝 HTML → Markdown 변환 시작
✅ Markdown 변환 완료 (길이: 15,670 문자)
📋 현재 단계: parsing_md
📄 콘텐츠 요약 시작
✅ 요약 완료 (길이: 459 문자)
📝 요약 내용: - 카카오는 AI 기반의 운영 생태계를 구축하기 위해 매트릭스(MATRIX)라는 모니터링 솔루션을 개발하여 서버 및 애플리케이션의 성능을 실시간으로 모니터링하고 있습니다.
- 매트...
📋 현재 단계: summarize
🏷️ 태그 생성 시작
✅ 태그 생성 완료: ['AI 운영', '분산 추적', '모니터링 솔루션', 'MATRIX', 'LLM', '인시던트 관리', '카카오 기술']
📋 현재 단계: tagging
📊 요약 품질 평가 시작
✅ 요약 평가 완료: 85점
📝 평가 이유: 요약은 원본의 핵심 내용을 잘 반영하고 있으며, 정확성과 간결성에서도 높은 점수를 받을 수 있습니다. 그러나 일부 세부 사항이 생략되어 있어 완성도와 구조화 측면에서 약간의 개선이 필요합니다.
📋 현재 단계: evaluate_summary
🏷️ 태깅 품질 평가 시작
✅ 태깅 평가 완료: 85점
📝 평가 이유: 태그는 원본 콘텐츠의 주제와 관련성이 높고, 주요 개념들을 잘 반영하고 있습니다. 태그의 개수와 구체성도 적절하며, 검색 및 분류에 유용합니다. 그러나 '카카오 기술'과 같은 태그는 다소 일반적이어서 다양성 측면에서 아쉬움이 있습니다.
📊 평가 결과 - 요약: 85점, 태깅: 85점
📋 현재 단계: evaluate_tagging
🎉 워크플로우 성공 완료! (요약: 85점, 태깅: 85점)
📋 현재 단계: finalize
🎯 최종 결과:
   • URL: https://tech.kakao.com/po

### 테스트 실행 예시

다음 셀을 실행하여 AI Agent 워크플로우를 테스트해보세요.

In [ ]:
# 개선된 AI Agent 테스트 실행
print("🧪 개선된 AI Agent 워크플로우 테스트")
print("=" * 50)
print("변경 사항:")
print("• 요약과 태깅을 별도로 평가 (각각 0-100점)")
print("• 두 점수 모두 80점 이상이어야 성공")
print("• Pydantic 모델을 사용한 구조화된 평가")
print("• LLM 기반 평가 (휴리스틱 제거)")
print("=" * 50)

🧪 개선된 AI Agent 워크플로우 테스트
변경 사항:
• 요약과 태깅을 별도로 평가 (각각 0-100점)
• 두 점수 모두 80점 이상이어야 성공
• Pydantic 모델을 사용한 구조화된 평가
• LLM 기반 평가 (휴리스틱 제거)


## ✅ 구현 완료!

인프런 AI 에이전트 강의를 참고하여 `ai_agent_workflow.md`에 정의된 워크플로우를 성공적으로 구현했습니다.

### 구현된 기능:
1. **get_html**: Playwright를 사용한 웹 페이지 로딩
2. **parsing_md**: HTML → Markdown 변환 
3. **요약**: LLM을 활용한 핵심 내용 추출
4. **태깅**: 자동 태그/카테고리 생성
5. **점수 평가**: 콘텐츠 품질 평가 (70점 기준)
6. **LangGraph 워크플로우**: StateGraph로 전체 플로우 관리
7. **재시도 로직**: 점수 미달 시 최대 3회 재시도

### 사용 방법:
```python
# 단일 URL 테스트
result = await run_ai_agent("https://example.com")

# 여러 URL 테스트
results = await test_ai_agent()
```

### 워크플로우 흐름:
```mermaid
flowchart TD
    START([시작]) --> URL[URL 입력]
    URL --> HTML[get_html<br/>웹 페이지 로딩]
    HTML --> PARSE[parsing_md<br/>HTML → Markdown 변환]
    
    PARSE --> SUMMARY[요약<br/>핵심 내용 추출]
    PARSE --> TAGGING[tagging<br/>카테고리/태그 생성]
    
    SUMMARY --> SCORE[점수<br/>품질 평가]
    TAGGING --> SCORE
    
    SCORE --> DECISION{점수 >= 70점?}
    DECISION -->|Yes| SUCCESS([성공 완료])
    DECISION -->|No| RETRY{재시도 < 3회?}
    
    RETRY -->|Yes| PARSE
    RETRY -->|No| FAIL([실패])
```